In [1]:
#%pip install optuna-integration[lightgbm]

import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
from optuna.integration import LightGBMPruningCallback
import sqlite3

/home/pablo/anaconda3/envs/LaboIII/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')

In [ ]:
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
# Reunir los DataFrames df_full y df_pendientes por PRODUCT_ID, CUSTOMER_ID y PERIODO, agregar las 
# columnas de df_pendientes a df_full
df_full = df_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))
# Imprimir las columnas de df_full
print("Columnas de df_full después de la unión con df_pendientes:")
print(df_full.columns.tolist())

In [ ]:
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = df_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908, 201910] else 0)

In [ ]:
# Optimizar tipos de datos numéricos
for col in df_full.select_dtypes(include=['int64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='integer')
for col in df_full.select_dtypes(include=['float64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='float')
# Variables categóricas
# categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
categorical_features = ['ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','PLAN_PRECIOS_CUIDADOS','MES_PROBLEMATICO','A_PREDECIR']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')

In [ ]:
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE' y 'CLASE_DELTA'
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=['CLASE', 'CLASE_DELTA']) 
# Filtrar en y que el periodo sea menor o igual a 201910
y = df_full[df_full['PERIODO'] <= 201910]['CLASE_DELTA']
# Eliminar df_full para liberar memoria
del df_full
gc.collect()

In [ ]:
# Definir los periodos de validación 201910
#periodos_valid = [201910]
periodos_valid = [201910]

# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train = y[X['PERIODO'] < periodos_valid[0]]
X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_list = [y[X['PERIODO'] == p] for p in periodos_valid]
del X, y
gc.collect()

In [ ]:
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback
import os

# === Usamos solo el primer período de validación ===
X_val = X_val_list[0]
y_val = y_val_list[0]

train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features)

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',  # alias de l1
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 31, 1024, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 24),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),
        'verbose': -1,
        'feature_pre_filter': False,
        'bagging_seed': 42,
        'feature_fraction_seed': 42
    }

    model = lgb.train(
        params,
        train_data,
        num_boost_round=2000,
        valid_sets=[val_data],  # No usar valid_names
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=100),
            LightGBMPruningCallback(trial, 'l1')  # Este es el nombre correcto
        ]
    )

    best_score = model.best_score['valid_0']['l1']
    print(f"Trial {trial.number}: MAE = {best_score:.5f}")
    return best_score

# Optuna
storage_url = "sqlite:///./modelos/optuna.db"
study = optuna.create_study(
    direction='minimize',
    study_name="mae_delta_lgbm_regression_todos_los_productos_con_pendientes",
    storage=storage_url,
    load_if_exists=True
)
study.optimize(objective, n_trials=200, show_progress_bar=True)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(study.best_params)

# Entrenamiento final
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'mae'
best_params['verbose'] = -1

model_reg = lgb.train(
    best_params,
    train_data,
    num_boost_round=50000,
    valid_sets=[val_data],
    callbacks=[
        lgb.early_stopping(stopping_rounds=500),
        lgb.log_evaluation(period=500)
    ]
)

os.makedirs('./modelos', exist_ok=True)
model_reg.save_model('./modelos/lgbm_model_reg_todos_los_productos.txt')



In [ ]:
# Obtener la importancia de cada variable
importancia = model_reg.feature_importance(importance_type='gain')
nombres = X_train.columns

# Crear un DataFrame ordenado por importancia
df_importancia = pd.DataFrame({'feature': nombres, 'importance': importancia})
df_importancia = df_importancia.sort_values(by='importance', ascending=False)

# Mostrar las variables más importantes
print(df_importancia.head(20))


# Si quieres visualizarlo gráficamente:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.barh(df_importancia['feature'], df_importancia['importance'])
plt.gca().invert_yaxis()
plt.title('Importancia de variables LightGBM')
plt.xlabel('Importancia')
plt.show()

In [2]:
# Entreno nuevamente el modelo con los mejores hiperparámetros y el conjunto completo de datos
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
# Reunir los DataFrames df_full y df_pendientes por PRODUCT_ID, CUSTOMER_ID y PERIODO, agregar las 
# columnas de df_pendientes a df_full
df_full = df_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = df_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908] else 0)
# Optimizar tipos de datos numéricos
for col in df_full.select_dtypes(include=['int64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='integer')
for col in df_full.select_dtypes(include=['float64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='float')
# Variables categóricas
# categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
categorical_features = ['ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','PLAN_PRECIOS_CUIDADOS','MES_PROBLEMATICO','A_PREDECIR']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE' y 'CLASE_DELTA'
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=['CLASE', 'CLASE_DELTA']) 
# Filtrar en y que el periodo sea menor o igual a 201910
y = df_full[df_full['PERIODO'] <= 201910]['CLASE_DELTA']
# Eliminar df_full para liberar memoria
del df_full
gc.collect()
# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X
y_train = y
del X, y
gc.collect()

0

In [3]:
# Obtener los mejores hiperparámetros del estudio de la base de datos SQLite
storage_url = "sqlite:///./modelos/optuna.db"
study1 = optuna.load_study(
    study_name="mae_delta_lgbm_regression_todos_los_productos_con_pendientes",
    storage=storage_url
)
best_params1 = study1.best_params
study1.best_params

{'num_leaves': 159,
 'learning_rate': 0.15976562141122164,
 'feature_fraction': 0.9902705425545245,
 'bagging_fraction': 0.9641034133841936,
 'bagging_freq': 3,
 'min_data_in_leaf': 311,
 'max_depth': 18,
 'lambda_l1': 2.4322841274280815,
 'lambda_l2': 0.4206330467495403,
 'min_gain_to_split': 0.9020185871164503}

In [ ]:

train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)

# Entrenamiento final
best_params = study1.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'mae'
best_params['verbose'] = -1

model_reg = lgb.train(
    best_params,
    train_data,
    num_boost_round=1000 
)

os.makedirs('./modelos', exist_ok=True)
model_reg.save_model('./modelos/lgbm_model_reg_todos_los_productos.txt')

In [ ]:
del X_train, y_train
gc.collect()

In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones

gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')
df_full = df_full[df_full['PERIODO'] == 201910].drop(columns=['CLASE', 'CLASE_DELTA'])
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
df_pendientes = df_pendientes[df_pendientes['PERIODO'] == 201910]
df_pred_full = df_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))
# Filtrar solo los que tengan la columna A_PREDECIR con valor 1
df_pred_full = df_pred_full[df_pred_full['A_PREDECIR'] == 1]
# Agregar a df_pred_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_pred_full['MES_PROBLEMATICO'] = df_pred_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908] else 0)
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_pred_full[col] = df_pred_full[col].astype('category')

In [ ]:
# Eliminar del dataframe df_pred_full la columna 'PREDICCIONES'
if 'PREDICCIONES' in df_pred_full.columns:
    df_pred_full.drop(columns=['PREDICCIONES'], inplace=True)

In [ ]:

predictions = model_reg.predict(df_pred_full) 
df_pred_full['PREDICCIONES'] = predictions


In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de las predicciones y la suma de la clase observada
df_comp = df_pred_full.groupby('PRODUCT_ID').agg({'PREDICCIONES': 'sum', 'CLASE': 'sum'}).reset_index()
df_comp['DIF_ABS'] = np.abs(df_comp['PREDICCIONES'] - df_comp['CLASE'])
# ordernar por la diferencia absoluta
df_comp = df_comp.sort_values(by='DIF_ABS', ascending=False)
df_comp.head(20)


In [ ]:
# Para los primeros 30 PRODUCT_ID de df_comp generar graficos como el anterior agregando resaltado en el gráfico en el periodo 201912
# el valor PREDICCIONES de DF_COMP para el PRODUCT_ID en el título del gráfico
for product_id in df_comp['PRODUCT_ID'].head(30):
    df_prod = df_full[df_full['PRODUCT_ID'] == product_id].groupby('PERIODO').agg({'TN': 'sum'}).reset_index()
    df_prod['PERIODO'] = df_prod['PERIODO'].astype(str)
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df_prod, x='PERIODO', y='TN', marker='o')
    pred_value = df_comp[df_comp["PRODUCT_ID"] == product_id]["PREDICCIONES"].values[0]
    dif = df_comp[df_comp["PRODUCT_ID"] == product_id]["DIF_ABS"].values[0]
    plt.title(f'Evolución de las TN para PRODUCT_ID {product_id} - Predicciones: {pred_value:.2f} - Dif Abs: {dif:.2f}')
    plt.xlabel('Periodo')
    plt.ylabel('Total de TN')
    plt.xticks(rotation=45)
    plt.ylim(0, df_prod['TN'].max() * 1.1)  # Ajustar el eje y para que comience en cero
    plt.scatter('201912', pred_value, color='red', s=100, zorder=5, label='Predicho')
    plt.legend()
    plt.grid()
    plt.show()



In [ ]:
# Generar el DataFrame de salida que contiene PRODUCT_ID y la suma de las predicciones por PRODUCT_ID
df_output = df_pred_full.groupby('PRODUCT_ID')['PREDICCIONES'].sum().reset_index()
# Hacer que la columna de predicciones sea mayor que cero
df_output['PREDICCIONES'] = np.where(df_output['PREDICCIONES'] < 0, 0, df_output['PREDICCIONES'])
df_output.head(10)
# Renombrar las columnas como product_id y tn
df_output.columns = ['product_id', 'tn']
# Guardar el DataFrame de salida en un archivo CSV
df_output.to_csv('./modelos/optuna_lgbm_predictions_sin_val_sin_negativos.csv', index=False)
# Contar los valores negativos en df_output
negativos = df_output[df_output['tn'] < 0].shape[0]
print(f"Número de valores negativos en las predicciones: {negativos}")